# Heatmap (Grad Cam) and Localization



In [11]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import os
import cv2

In [12]:
input_shape = (99,2792,1)
image_shape = (2792,99)

output_layer_num = -4

WHEEL_PART = "ajours"

DATA_PATH = "/home/cogrannr/roues/MEFRO/grises/img_"+WHEEL_PART+"_avec_defauts/"
MASK_PATH = "/home/cogrannr/roues/MEFRO/grises/img_"+WHEEL_PART+"_masque_defauts/"

MODEL_RESNET = "/home/etu/derouetl/mefro_v2/trained_models/wheel_part/gray/resnet_gray_"+WHEEL_PART+".h5"

threshold = 0.7

# Defauts réels


In [13]:
class GradCam():

    def __init__(self, model_name, nb_layers,
        input_shape=(99,2792,1), image_shape= (2792,99)):

        self.model_name = model_name
        self.input_shape = input_shape
        self.image_shape = image_shape
        self.nb_layers = nb_layers
        self.model = self.init_model(nb_layers, model_name)
        self.saved_cam = []
        self.file_names_list = []

    def load_image(self, name):

        image = PIL.Image.open(name)
        image = image.resize(image_shape)
        image = np.array(image, dtype=np.float32)
        image = image[np.newaxis,:,:,np.newaxis]

        return image

    def load_image_rgb(self, name, noise_factor=7):

        image = self.load_image(name)
        image = np.repeat(image, 3, axis=3)
        #image = image + np.random.randn(image.shape)

        image = image + tf.random.normal(image.shape)*noise_factor
        image = tf.clip_by_value(image,0,255)

        return image

    def init_model(self, nb_layers, model_name):

        base_model = tf.keras.models.load_model(model_name)
        model_output = base_model.layers[nb_layers].output


        model = tf.keras.Model(inputs=base_model.input, outputs=[model_output, base_model.output])

        print(base_model.summary())
        print("model output ",base_model.layers[nb_layers])

        return model

    def heatmap(self, image, image_name, plot_images=False):

        with tf.GradientTape() as g:
            activation, pred = self.model(image)

        if pred == 1:
            #print("error : 0 gradient")
            return 1
        if pred < 0.5:
            #print(" pred : no defect")
            return 1

        d = g.gradient(pred, activation)

        d = tf.math.reduce_mean(d[0],axis=(0,1))
        activation = activation[0]

        cam = np.zeros(activation.shape[0:2])

        for i in range(0,activation.shape[2]):
            cam = cam + d[i]*activation[:,:,i]

        cam_relu = tf.math.maximum(cam, 0)
        cam_relu = tf.get_static_value(cam_relu)

        # appliquer cmap
        cam_relu = cv2.resize(cam_relu, (image.shape[2],image.shape[1]))

        cam_relu = cam_relu / (np.max(cam_relu)+ 1e-5)

        cam = np.uint8(255*cam_relu)

        self.saved_cam.append(cam)
        self.file_names_list.append(image_name)

        if plot_images == True:
            plt.matshow(cam)
            plt.show()
            input("press enter")

        return 0

    def heatmap_list_images(self, path, rgb=False, plot_images=False):

        for f in np.sort(os.listdir(path)):

            if rgb == False:
                image = self.load_image(path+f)
            else:
                image = self.load_image_rgb(path+f)

            #print("current image", f)
            self.heatmap(image, f, plot_images=plot_images)

    def get_grad_cam(self):
        cam = self.saved_cam
        cam = np.array([*cam])
        self.saved_cam = []
        return cam

    def get_file_names_list(self):
        files = self.file_names_list
        self.file_names_list = []
        return files

    def save_grad_cam(self, name):

        computed_cam = np.array([*self.saved_cam])
        file_names = np.array(self.file_names_list)

        np.savez_compressed(name,grad_cam=computed_cam, file_name=file_names)


In [14]:
class Localization_from_heatmap():
    def __init__(self, threshold=0.7):
        self.heatmaps = []
        self.bbox_list = []
        self.threshold = threshold

    def init_data(self, data_intput):

        # just one heatmap (n,m) make it (1,h,w)
        if data_intput.ndim == 2:
            self.heatmaps = data_intput[nb.newaxis,:,:]

        # heatmap (n,h,w)
        if data_intput.ndim == 3:
            self.heatmaps = data_intput


    def init_data_from_file(self, file_name):
        data = np.load(file_name)
        heatmaps = data[data.files[0]]
        file_names = data[data.files[1]]
        return heatmaps, file_names

    def threshold_heatmaps(self):
        for i in range(self.heatmaps.shape[0]):
            array = self.heatmaps[i]
            array[ array < self.threshold*array.max() ] = 0
            array[array != 0] = 255

            self.heatmaps[i] = array

    def compute_bounding_box(self, data):
        self.init_data(data)

        self.threshold_heatmaps()

        for i in range(self.heatmaps.shape[0]):
            contour = cv2.findContours(self.heatmaps[i], cv2.RETR_TREE,
                cv2.CHAIN_APPROX_SIMPLE)[0]

            if len(contour) > 1:
                return 1
                
            contour = contour[0]

            #x,y  : top left coordinate, horizontal axis
            # width and height horiz and vertical
            x,y,w,h = cv2.boundingRect(contour)
            self.bbox_list.append([x,y,w,h])
            
            return 0

    def get_bounding_box(self):
        bbox = self.bbox_list
        self.bbox_list = []
        self.heatmaps = []
        return self.adapt_bbox(bbox)

    def adapt_bbox(self, bbox_list):

        adapted_bbox = []

        for box in bbox_list:

            top_left_corner_x = box[1]
            top_left_corner_y = box[0]

            h = box[3]
            w = box[2]

            x = top_left_corner_x + h//2
            y = top_left_corner_y + w//2

            adapted_bbox.append([x,y,h,w])

        return np.array(adapted_bbox)

    def plot_gray_images_with_bbox(self, image_name_list, image_path, bbox_list, heamap_list, image_shape=(2792,99)):

        b = self.adapt_bbox(bbox_list)

        for i in range(len(image_name_list)):
            print(image_name_list[i])
            print(b[i])

            image_name = image_name_list[i]

            image = PIL.Image.open(image_path+image_name)
            image = image.resize(image_shape)
            image = np.array(image, dtype=np.uint8)
            image = image[:,:,np.newaxis]
            image = np.repeat(image, 3, axis=2)

            bbox = bbox_list[i]

            image = cv2.rectangle(image, (bbox[0], bbox[1]),
                (bbox[0]+bbox[2], bbox[1]+bbox[3]), (255,0,0), 3)

            #cv2 draw rect puis plot
            fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20,2), sharex=True)#,sharex=True, gridspec_kw={'hspace': 10})

            axs[0].imshow(image)
            axs[1].imshow(heamap_list[i])

            plt.show()

            input("next image?")

In [15]:
def generate_bbox_from_mask(list_files, path_mask="", image_shape=(2792,99)):
    bbox_mask = []
    i=0
    for file in list_files:
        
        if i%100==0:
            print(i)
        file = file[0]
        image = PIL.Image.open(path_mask+file[:-3]+"png")
        image = np.array(image.resize(image_shape))

        indexes = np.where(image == 255)

        top_left_corner = [ indexes[0][0], indexes[1][0] ]

        bottom_right_corner = [ indexes[0][-1], indexes[1][-1] ]
        # horizontald length
        bbox_w  = bottom_right_corner[1] - top_left_corner[1] +1
        # vertical length
        bbox_h = bottom_right_corner[0] - top_left_corner[0] +1
        # center x coordinate
        bbox_x = top_left_corner[0] + bbox_h//2
        # center y coordinate
        bbox_y = top_left_corner[1] + bbox_w//2

        bbox_mask.append(np.array([bbox_x, bbox_y, bbox_h, bbox_w]))
        i=i+1

    return np.array([*bbox_mask])

In [16]:
def compute_IoU(predicted_bbox, ground_truth_bbox):

    pred_x1 = predicted_bbox[:,0] - predicted_bbox[:,2]//2
    pred_x2 = predicted_bbox[:,0] + predicted_bbox[:,2]//2
    pred_y1 = predicted_bbox[:,1] - predicted_bbox[:,3]//2
    pred_y2 = predicted_bbox[:,1] + predicted_bbox[:,3]//2

    truth_x1 = ground_truth_bbox[:,0] - ground_truth_bbox[:,2]//2
    truth_x2 = ground_truth_bbox[:,0] + ground_truth_bbox[:,2]//2
    truth_y1 = ground_truth_bbox[:,1] - ground_truth_bbox[:,3]//2
    truth_y2 = ground_truth_bbox[:,1] + ground_truth_bbox[:,3]//2

    I_x1 = np.maximum(pred_x1, truth_x1)
    I_y1 = np.maximum(pred_y1, truth_y1)

    I_x2 = np.minimum(pred_x2, truth_x2)
    I_y2 = np.minimum(pred_y2, truth_y2)

    intersection = np.maximum((I_x2 - I_x1 + 1),0)*np.maximum((I_y2 - I_y1 +1),0)

    IoU = intersection / ( (truth_y2 - truth_y1+1)*(truth_x2 - truth_x1+1) +
            (pred_x2 - pred_x1+1)*(pred_y2 - pred_y1+1) - intersection)

    return IoU

In [17]:
# compute 1000 bbox

grad_cam_tool = GradCam(MODEL_RESNET, output_layer_num)
localization_tool = Localization_from_heatmap()

image_names = os.listdir(DATA_PATH)

i=0

grad_cam_file_name_list = []

while len(localization_tool.bbox_list) != 2000:
    
    if i%50==0:
        print("image ",i, "grad cam : ",len(localization_tool.bbox_list))
    
    image = grad_cam_tool.load_image_rgb(DATA_PATH+image_names[i])

    result = grad_cam_tool.heatmap(image,image_names[i])

    if result == 0:
        grad_cam = grad_cam_tool.get_grad_cam()
        
        r = localization_tool.compute_bounding_box(grad_cam)
    
        if r == 0:
            grad_cam_file_name = grad_cam_tool.get_file_names_list()
            grad_cam_file_name_list.append(grad_cam_file_name)
            
    i=i+1

bbox_arr = localization_tool.get_bounding_box()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 99, 2792, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 105, 2798, 3) 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 1396, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 52, 1398, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

image  0 grad cam :  0
image  50 grad cam :  5
image  100 grad cam :  11
image  150 grad cam :  19
image  200 grad cam :  25
image  250 grad cam :  31
image  300 grad cam :  38
image  350 grad cam :  44
image  400 grad cam :  50
image  450 grad cam :  56
image  500 grad cam :  60
image  550 grad cam :  62
image  600 grad cam :  71
image  650 grad cam :  76
image  700 grad cam :  80
image  750 grad cam :  89
image  800 grad cam :  94
image  850 grad cam :  104
image  900 grad cam :  113
image  950 grad cam :  123
image  1000 grad cam :  129
image  1050 grad cam :  131
image  1100 grad cam :  135
image  1150 grad cam :  136
image  1200 grad cam :  144
image  1250 grad cam :  151
image  1300 grad cam :  160
image  1350 grad cam :  166
image  1400 grad cam :  175
image  1450 grad cam :  181
image  1500 grad cam :  187
image  1550 grad cam :  195
image  1600 grad cam :  201
image  1650 grad cam :  210
image  1700 grad cam :  218
image  1750 grad cam :  228
image  1800 grad cam :  231
image 

KeyboardInterrupt: 

In [18]:
bbox_mask=generate_bbox_from_mask(grad_cam_file_name_list,MASK_PATH)

np.savez_compressed("localization_pred_and_mask_ajours_2",bbox_pred=bbox_arr,
                    file_names=np.array(grad_cam_file_name_list), bbox_mask=bbox_mask)

IoU=compute_IoU(bbox_arr, bbox_mask)
np.savez_compressed("IoU_ajours_2",IoU=IoU)

0
100
200
300
400
500
600
700
800
900
1000


<ipython-input-18-a5bb989189bd>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  file_names=np.array(grad_cam_file_name_list), bbox_mask=bbox_mask)
